In [ ]:
import torch
from torch.utils.data import DataLoader

import evaluate
from args import args, config
from items_dataset import items_dataset
from models import Model_Crf, Model_Softmax

In [ ]:
test_loader  = torch.load(args.DATA_PATH + "test.pt")
#test_loader = DataLoader(test_set, batch_size=args.batch_size, shuffle=False, collate_fn=test_set.test_collate_fn)

In [ ]:
directory = "../model/"
model_name = "roberta_crf_epoch_1.pt"
device = torch.device('cuda', 0) if torch.cuda.is_available() else torch.device('cpu')

if args.use_crf:
    model = Model_Crf(config).to(device)
else:
    model = Model_Softmax(config).to(device)
model.load_state_dict(state_dict=torch.load(directory + model_name, map_location=device))

In [ ]:
predict_list, ground_truth, text_list = evaluate.test_encode_predict(test_loader, device, model)

In [ ]:
"""
print("Charecter based scores for check_worthy:")
all_confusion_matrix = dict()
all_scores = dict()
for metric in ground_truth_dict.keys():
  confusion_matrix = evaluate.cal_span_level_matrix(predict_list, ground_truth_dict[metric], text_list)
  scores = evaluate.calculate_precision_recall_f1(confusion_matrix)
  all_confusion_matrix[metric] = confusion_matrix
  all_scores[metric] = scores
  print(metric)
  print(confusion_matrix)
  print(scores, end ="\n\n")
"""

In [ ]:
#"Charecter based scores for majority vote:"
confusion_matrix = evaluate.cal_span_level_matrix(predict_list, ground_truth, text_list)
print(confusion_matrix)
#print("check_worthy", evaluate.calculate_precision_recall_f1(confusion_matrix))
check_worthy = evaluate.calculate_precision_recall_f1(confusion_matrix)
print("check_worthy", check_worthy)

confusion_matrix_non_worthy = confusion_matrix.copy()
tmp = confusion_matrix_non_worthy["TP"]
confusion_matrix_non_worthy["TP"] = confusion_matrix_non_worthy["TN"]
confusion_matrix_non_worthy["TN"] = tmp
tmp = confusion_matrix_non_worthy["FN"]
confusion_matrix_non_worthy["FN"] = confusion_matrix_non_worthy["FP"]
confusion_matrix_non_worthy["FP"] = tmp
non_check_worthy = evaluate.calculate_precision_recall_f1(confusion_matrix_non_worthy)
print("non_check_worthy", non_check_worthy)


In [ ]:
print("macro F1: %.4f" %((check_worthy["f1"]+non_check_worthy ["f1"])/2))
print("macro Precision: %.4f" %((check_worthy["precision"]+non_check_worthy ["precision"])/2))
print("macro Recall: %.4f" %((check_worthy["recall"]+non_check_worthy ["recall"])/2))

print("    Checkworthy, Non_check_worthy")
print("F1: %.4f,         %.4f" %(check_worthy["f1"], non_check_worthy ["f1"]))
print("Precision: %.4f,         %.4f" %(check_worthy["precision"], non_check_worthy ["precision"]))
print("Recall: %.4f,         %.4f" %(check_worthy["recall"], non_check_worthy ["recall"]))

print("Acc: %.4f" %check_worthy["accuracy"])